## 线性回归的解析方法

### 1. 模型

输入向量与标签：
$$
X = \begin{bmatrix}x^T_1 \\ x^T_2\\ \vdots \\ x^T_N\end{bmatrix}, Y = \begin{bmatrix}y_1 \\ y_2\\ \vdots \\ y_N\end{bmatrix}
$$

损失函数（均方误差）：

$$
J(\theta) = \frac{1}{2N}(y-X\theta)^T(y-X\theta)
$$

解析解（导数为0）：
$$
\theta = (X^TX)^{-1}X^Ty
$$

模型对训练数据的预测：

$$
f(\theta) = X\theta = X(X^TX)^{-1}X^Ty
$$

### 2. 代码实现